# HDBSCAN should work well on embedding representations

In [2]:
import os
import re
import json
import hdbscan

import numpy as np
import pandas as pd

from sklearn.decomposition import PCA
from collections import Counter
import matplotlib.pyplot as plt

import lib.helper as helper
import lib.embedding_models as reps

from importlib import reload

%matplotlib inline

## 1.  Retrieve Corpus

The corpus is being scraped by the "run_news_scrapes.py" script (and windows task scheduler) every 12 hours, a bit past midday and a bit past midnight.

The "bing" corpus are news titles and text extracts gotten from the bing news search API, using a few Home Office - related keywords.

The "RSS" corpus is plugged directly into a number of RSS feeds for world news sites and local british news sites, with no filters for news story types or subjects applied.

In [25]:
# Should be same path for all my PC's, it's where each scrape goes as a separate json file.
storage_path = "D:/Dropbox/news_crow/scrape_results"

# "bing" is targeted news search corpus, "RSS" is from specific world and local news feeds.
corpus_type = "RSS"

# Load up
corpus = helper.load_clean_corpus(storage_path, corpus_type)

# Make sure after cleaning etc it's indexed from 0
corpus.reset_index(inplace=True)
corpus.index.name = "node"

# See how it turned out
print(corpus.shape)
corpus.head()

Total files: 318
9.7 percent of files read.
19.5 percent of files read.
29.2 percent of files read.
39.0 percent of files read.
48.7 percent of files read.
58.5 percent of files read.
68.2 percent of files read.
78.0 percent of files read.
87.7 percent of files read.
97.5 percent of files read.
(78055, 9)


,index,title,summary,date,link,source_url,retrieval_timestamp,origin,clean_text
node,,,,,,,,,
0,0,Hurricane Dorian lashes US as Bahamas counts cost,"Life-threatening US storm surges are feared, a...","Thu, 05 Sep 2019 16:03:44 GMT",https://www.bbc.co.uk/news/world-us-canada-495...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Hurricane Dorian lashes US as Bahamas counts c...
1,1,Kohistan video murders: Three guilty in 'honou...,They are relatives of a group of Pakistani wom...,"Thu, 05 Sep 2019 13:53:17 GMT",https://www.bbc.co.uk/news/world-asia-49592540,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Kohistan video murders: Three guilty in 'honou...
2,2,MH17 Ukraine plane crash: 'Key witness' released,A Ukrainian court releases a potentially key w...,"Thu, 05 Sep 2019 13:46:06 GMT",https://www.bbc.co.uk/news/world-europe-49591148,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,MH17 Ukraine plane crash: 'Key witness' releas...
3,3,Article 370: The weddings 'ruined' by Kashmir'...,Indian-administered Kashmir is under a securit...,"Thu, 05 Sep 2019 07:32:34 GMT",https://www.bbc.co.uk/news/world-asia-india-49...,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Article 70: The weddings 'ruined' by Kashmir's...
4,4,Syria war: Turkey warns Europe of new migrant ...,President Erdogan demands international help t...,"Thu, 05 Sep 2019 16:11:48 GMT",https://www.bbc.co.uk/news/world-europe-49599297,http://feeds.bbci.co.uk/news/world/rss.xml,2019-09-05 21:35:06.925873,rss_feed,Syria war: Turkey warns Europe of new migrant ...


## 2.  Build Text Model (Representation, eg; word2vec, entities list...)

- Trying with the world corpus and with the bing corpus, neither worked with InferSent.  Suspect the problem lies in the PCA step, which may not be working well on this high-dimensional (vector length = 4096) form.
- Summed keywords works rather better with the world corpus.
- Summed keywords still fail the bing/home office corpus, giving me a cluster about "immigration" and a cluster for the American Supreme Court.

In [5]:
# Windows didn't play nicely with the vector datasets, Some obscure encoding problem (python in Conda
# kept trying to decode using cp1252 regardless of whatever other options I specified!)
# Solution; rewrite file and drop any characters the Windows encoder refuses to recognise.
# I shouldn't loose too much info.
with open('./lib/InferSent/dataset/fastText/crawl-300d-2M.vec', "r", encoding="cp1252", errors="ignore") as infile:
    with open('./lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec', "wb") as outfile:
        for line in infile:
            outfile.write(line.encode('cp1252'))

In [117]:
infersent = reps.InferSentModel(list(corpus['clean_text']),
                                list(corpus['clean_text']),
                                W2V_PATH = './lib/InferSent/dataset/fastText/crawl-300d-2M_win.vec')

embeddings = infersent.get_embeddings()

Found 16344(/17591) words with w2v vectors
Vocab size : 16344


In [26]:
# Whereas this worked first time!
glove = reps.GloveWordModel(list(corpus['clean_text']), list(corpus['clean_text']))

embeddings = glove.get_embeddings()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Martin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
# Turn that into a DF for me
embeddings_df = pd.DataFrame({"clean_text": list(embeddings.keys()),
                              "embeddings": list(embeddings.values())})
embeddings_df.shape

(78042, 2)

In [28]:
embeddings_df.head()

,clean_text,embeddings
0,Hurricane Dorian lashes US as Bahamas counts c...,"[0.37263061209893433, -0.06016781549791428, 0...."
1,Kohistan video murders: Three guilty in 'honou...,"[0.20238062166675427, -0.03851285659736075, 0...."
2,MH17 Ukraine plane crash: 'Key witness' releas...,"[0.2929107795220376, -0.0020010595910374285, 0..."
3,Article 70: The weddings 'ruined' by Kashmir's...,"[-0.05375611008834829, -0.032535791358411996, ..."
4,Syria war: Turkey warns Europe of new migrant ...,"[0.25193344012630575, 0.11297337160719789, -0...."


## 3. Cluster Text

This is the part where the pipelines get a little more experimental

In [29]:
embeddings_array = np.vstack(embeddings_df['embeddings'])

In [30]:
embeddings_array.shape

(78042, 50)

In [31]:
# First, PCA the data
pca = PCA(n_components=20, svd_solver='full')

embeddings_pca = pca.fit_transform(embeddings_array)

In [32]:
embeddings_pca.shape

(78042, 20)

In [33]:
print(pca.explained_variance_ratio_)
print(pca.singular_values_) 

[0.13159809 0.10506169 0.09024923 0.05968243 0.05524087 0.0406593
 0.03872906 0.03245727 0.03023115 0.0267074  0.02648908 0.02266101
 0.02166204 0.01882934 0.01786638 0.01761238 0.01669234 0.01501542
 0.01414985 0.01378657]
[136.60343793 122.0559702  113.12499867  91.99412146  88.50485123
  75.9305886   74.10633183  67.84109675  65.47328688  61.53931765
  61.28727349  56.68609133  55.42255746  51.67189198  50.33327347
  49.97420727  48.65141519  46.14296626  44.79326907  44.21452885]


In [45]:
clusterer = hdbscan.HDBSCAN(min_cluster_size=2, min_samples=1)
clusterer.fit(embeddings_pca)
pd.unique(clusterer.labels_)

array([7283, 6942, 3440, ..., 9552,  337, 3846], dtype=int64)

In [46]:
len(pd.unique(clusterer.labels_))

12151

In [47]:
Counter(clusterer.labels_)

Counter({7283: 2,
         6942: 6,
         3440: 2,
         -1: 47417,
         7249: 3,
         8478: 2,
         706: 2,
         827: 3,
         6097: 2,
         3758: 4,
         2256: 3,
         5974: 2,
         8015: 3,
         1857: 2,
         1986: 2,
         10130: 2,
         4394: 3,
         6460: 3,
         8988: 3,
         1867: 2,
         4705: 3,
         8900: 2,
         7734: 2,
         10827: 3,
         5025: 2,
         6765: 5,
         1540: 2,
         4726: 3,
         790: 2,
         3435: 3,
         2106: 3,
         1733: 3,
         588: 2,
         8031: 2,
         9563: 2,
         4433: 2,
         7440: 6,
         235: 2,
         5933: 5,
         9567: 2,
         8412: 2,
         3358: 2,
         5719: 2,
         5523: 3,
         8044: 3,
         10591: 2,
         3101: 3,
         2108: 2,
         9624: 3,
         3441: 2,
         9748: 6,
         6409: 2,
         10976: 2,
         1233: 3,
         4867: 2,
         